In [3]:
'''
Generates frequent item sets and association rules from given datasets using Apriori algorithm.
'''
# pylint: disable=invalid-name
import csv
import itertools
import time
import matplotlib.pyplot as plt

DataFile = open('Market_Basket_Optimisation.csv', 'r')
# DataFile = open('groceries.csv', 'r')
DataCaptured = csv.reader(DataFile, delimiter=',')
data = list(DataCaptured)
print(len(data))
minsup = float(input("Support-Threshold: ")) #某物含的%數
f2 = "Apriori_Rules.txt"
f1 = "Apriori_frequent_pattern.txt"
minconf = float(input("Confidence-Threshold: "))
time_start = time.time()


def L1(data): #計算L1
    '''
    Find frequent 1-itemsets
    '''
    #Get all 1-itemsets in the list items and their counts in the dictionary counts
    row_len= len(data)
    count = {}
    for items in data:
        for item in items:
            if item not in count:                   #計算個別總量
                count[(item)] = 1
            else:
                count[(item)] += 1
    count2 = {k: v for k, v in count.items() if v >= minsup*row_len} #7501CSV行數，符合minsup我才要
    return count2


def generateCk(Lk_1, flag, data):
    '''
    Generate Ck by joining 2 Lk-1
    '''
    Ck = []
    # lk_1 = [a,b,c,e] 
    if flag == 1:
        flag = 0
        Ck += [(item1, item2) for item1 in Lk_1 for item2 in Lk_1 if item2 > item1]
    else:
        for item in Lk_1: #L2 = [('a', 'b'), ('a', 'c'), ('a', 'e'), ('b', 'c'), ('b', 'e'), ('c', 'e')]
            k = len(item)
        for item1 in Lk_1:
            for item2 in Lk_1:
                if (item1[:-1] == item2[:-1]) and (item1[-1] != item2[-1]): 
                    if item1[-1] > item2[-1]:
                        Ck.append(item2 + (item1[-1],)) #解決(a,b,b,c,)問題，指讓(a,b,c)存在 ，此外幣使用tuple才能產生('a','b')的key
                    else:                        
                        Ck.append(item1 + (item2[-1],)) #list是不行的
    L = generateLk(set(Ck), data)
    return L, flag


def generateLk(Ck, data):
    '''
    If item in Ck belongs to a transaction,
    it makes it into list Ct
    Then Ct is thresholded to form L
    '''
    row_len= len(data)
    count = {}
    for itemset in Ck:     #Ck = [(a,b),(a,c),(a,e), (b,c) ,(b,e),(c,e)]
        #print(itemset)
        for transaction in data: #data = list(DataCaptured)
            if all(e in transaction for e in itemset): # e = a , b itemset = (a,b)
                if itemset not in count: 
                    count[itemset] = 1
                else:
                    count[itemset] += 1
#     if ('ham', 'whole wheat rice') in count:
#         print("('ham', 'whole wheat rice')",count[('ham', 'whole wheat rice')])
    count2 = {k: v for k, v in count.items() if v >= minsup*row_len}
    return count2


def rulegenerator(fitems): #fitmes = 最終dic
    '''
    Generates association rules from the frequent itemsets
    '''
    counter = 0
    for itemset in fitems.keys():
        if isinstance(itemset, str): #只有單個得我不要，我要tuple形式('a','b')
            continue
        length = len(itemset)#tuple裡面有幾項
        union_support = fitems[tuple(itemset)]#原本itemset為LIST形式，必須使用tuple(才可以變成key ，看兩個以上的品項的出現次數
        for i in range(1, length):
            lefts = map(list, itertools.combinations(itemset, i)) #['mineral water'], ['pancakes']
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]#單品項出現次數 
                        conf = union_support / leftcount
                if conf >= minconf:
                    fo = open(f2, "a+")
                    right = list(itemset[:])
                    for e in left:                          #去除被分配在左邊的品項
                        right.remove(e)     
#                     fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')
                    print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
                    counter += 1
                    fo.close()
    print(counter, "rules generated")


def apriori():
    '''
    The runner function
    '''    
    L = L1(data)
    flag = 1
    FreqItems = dict(L)
    while(len(L) != 0): #算到字典數目不符合，minsup
#         print(L,len(L))
        fo = open(f1, "a+")
        for k, v in L.items():
            fo.write(str(k) + ' >>> ' + str(v) + '\n\n')
        fo.close()
        L, flag = generateCk(L, flag, data)
        FreqItems.update(L) #update = 字典的append
    print("Frequent_Patterns")
    print('number of frquent_pattern',len(FreqItems))
    print(FreqItems.keys())
    print("Association_Rules")
    rulegenerator(FreqItems)
    return list(FreqItems.keys())
#     plt.bar(range(len(FreqItems)), list(FreqItems.values()), align='center')
#     plt.xticks(range(len(FreqItems)), list(FreqItems.keys()))
#     plt.show()
if __name__ == '__main__':
    apriori()
    FreqItems = apriori()
time_end = time.time()
print('Apriori_spent_time', time_end - time_start)
#     print(time_end - time_start)

7501
Support-Threshold: 0.0045
Confidence-Threshold: 0.1
Frequent_Patterns
number of frquent_pattern 842
dict_keys(['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil', 'burgers', 'meatballs', 'eggs', 'turkey', 'milk', 'energy bar', 'whole wheat rice', 'whole wheat pasta', 'french fries', 'soup', 'light cream', 'shallot', 'frozen vegetables', 'spaghetti', 'pet food', 'cookies', 'cooking oil', 'champagne', 'chocolate', 'chicken', 'oil', 'fresh tuna', 'tomatoes', 'black tea', 'extra dark chocolate', 'protein bar', 'red wine', 'pasta', 'pepper', 'shampoo', 'rice', 'sparkling water', 'ham', 'body spray', 'pancakes', 'grated cheese', 'white wine', 'toothpaste', 'parmesan cheese', 'fresh bread', 'ground beef', 'escalope', 'herb & pepper', 'tomato sauce', 'magazines

['yogurt cake'] -> ['eggs'] (0.16585365853658537)
['parmesan cheese'] -> ['frozen vegetables'] (0.2751677852348993)
['low fat yogurt'] -> ['ground beef'] (0.1254355400696864)
['whole wheat pasta'] -> ['chocolate'] (0.21266968325791855)
['brownies'] -> ['green tea'] (0.1976284584980237)
['meatballs'] -> ['mineral water'] (0.36942675159235666)
['low fat yogurt'] -> ['french fries'] (0.17421602787456447)
['cereals'] -> ['pancakes'] (0.18134715025906736)
['oil'] -> ['mineral water'] (0.3179190751445087)
['tomato sauce'] -> ['mineral water'] (0.4056603773584906)
['ham'] -> ['eggs'] (0.2315270935960591)
['honey'] -> ['chocolate'] (0.16853932584269662)
['soup'] -> ['mineral water'] (0.45646437994722955)
['chicken'] -> ['ground beef'] (0.15777777777777777)
['cereals'] -> ['green tea'] (0.2538860103626943)
['black tea'] -> ['mineral water'] (0.37383177570093457)
['brownies'] -> ['pancakes'] (0.16205533596837945)
['nonfat milk'] -> ['mineral water'] (0.48717948717948717)
['cooking oil'] -> ['gra

['cooking oil'] -> ['mineral water', 'spaghetti'] (0.14882506527415143)
['milk'] -> ['chocolate', 'mineral water'] (0.10802469135802469)
['herb & pepper'] -> ['ground beef', 'mineral water'] (0.1347708894878706)
['ground beef'] -> ['mineral water', 'spaghetti'] (0.17367706919945725)
['chicken'] -> ['mineral water', 'spaghetti'] (0.11555555555555555)
['ground beef'] -> ['chocolate', 'mineral water'] (0.1112618724559023)
['olive oil'] -> ['milk', 'spaghetti'] (0.10931174089068826)
['olive oil'] -> ['chocolate', 'mineral water'] (0.12550607287449392)
['chocolate', 'mineral water'] -> ['olive oil'] (0.12550607287449392)
['chocolate', 'olive oil'] -> ['mineral water'] (0.12550607287449392)
['mineral water', 'olive oil'] -> ['chocolate'] (0.12550607287449392)
['salmon'] -> ['chocolate', 'mineral water'] (0.10658307210031348)
['chocolate', 'mineral water'] -> ['salmon'] (0.10658307210031348)
['chocolate', 'salmon'] -> ['mineral water'] (0.10658307210031348)
['mineral water', 'salmon'] -> ['ch

In [2]:
import csv
import pandas as pd
from itertools import combinations
import time


#Read data from CSV
# data = pd.read_csv('groceries.csv')
data = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
#
#Parameters
minsup = float(input("Support-Threshold: "))
minsup = minsup * len(data)
minconf = float(input("Confidence-Threshold: "))
print(len(data))#9835
time_start = time.time()

#Add all data in a list of lists
items = []  
# print(len(data.values[0]))#1列(ROW)有32的物品
for i in range(0, len(data)):  #將所有data中的物品一列一列放到items
     items.append([str(data.values[i, j]) for j in range(0, len(data.values[0]))])

#Creating a list of dictionaries
count = [dict() for x in range(len(data.values[0]) + 1)]
# print(items)
# #Count support for each individual items
s = []
for i in items:
    for j in i:      #讀取所有items
        s.append(j)
for i in s:                      #創字典 放在count[1]
    #If item is present in dictionary, increment its count by 1
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #If item is not present in dictionary, set its count to 1
    else:
        count[1][i] = 1     #{'i':1}

# #Remove infrequent and empty items #去除小於minsup,nan
for i in count[1].copy():
    if(count[1][i] < minsup):
        count[1].pop(i)
count[1].pop('nan') #去除以key = 'NAN'產生的值

# #Storing transactions as lists without infrequent items
a = list(count[1]) #列出所有符合的KEY
item = [list() for i in range(len(data))] #創造len(data) = 9835個list 
c = 0 
for i in range(0,len(items)): ##將所有data中的物品一列一列放到items
    for j in range(len(items[i])):
#         print(items[i][j])
        if(a.__contains__(items[i][j]) != 0): #確認物品是否存在
            item[i].append(items[i][j]) #將物品分成[['a', 'b', 'c'],[]]與itmes不同的地方是裡練已經沒有NAN了

#Function to sort list to support
def sort(a): #a = 所有符合的KEY的list
    for i in range(len(a) - 1):
        for j in range(len(a) - i - 1):
            if(count[1][a[j]] < count[1][a[j + 1]]): #如果下一個比上一個大則互換位子
                a[j],a[j + 1] = a[j + 1],a[j]
                
#Call function to sort all transactions in descending order of their support
for i in range(0,len(data)): #9835
    if(len(item[i]) > 1): #如果不只一個物件就排序他
        sort(item[i])

#Tree class for FP-Tree
class tree: #設計樹根，分支
    def __init__(self, name, sup, parent):
        self.name = name
        self.sup = sup
        self.nodeLink = None
        self.parent = parent
        self.children = []

#Function to check if the node is present is a child of the current node
def ispresent(node,name):    #確認是否有children
    f = -1               
    for i in node.children:
        f += 1
        if(i.name == name):
            return f
    return -1

#HeaderTable which stores the reference of last/first occurence of an item. Used as a linked list to generate candidate trees 
lastocc = count[1].copy() #符合sup的字典
for i in lastocc: #lastocc字典中所有值皆為NONE
    lastocc[i] = None
#Function to create FP-tree
root = tree("root", -1, None)
z = 0
for i in item: #item為沒有NAN的items
    current = root
    for j in range(len(i)): #len(i)9835
        if(ispresent(current,i[j]) >= 0): #第一次通常不符合此情形，直接執行else，看有沒有children
            current = current.children[ispresent(current, i[j])]
            current.sup = current.sup + 1
        else:
            child=tree(i[j], 1, current)    #創建i[j]子根
            current.children.append(child) #將子根放入children =[] 
            t = current #t 記錄柱上一層的tree1
            current = current.children[ispresent(current, i[j])] #現在current.children裡有子根，所以self.name = i[j]如下示意，
            #ispresent(current,i[j])回傳 0 ，現在的current為tree2
            current.parent = t #紀錄上一層tree1
            if(lastocc[current.name] == None):
                lastocc[current.name] = current #創建{'i[j]': tree2-1()} 
            else:#創出第一層並列樹枝
                current.nodeLink = lastocc[current.name] #將tree2-2連結到tree4
                lastocc[current.name] = current #CURRENT = TREE2-2 {'i[j]': tree2-2}
            
# class tree1:
#     def __init__(self, name, sup, parent):
#         self.name = 'root'
#         self.sup = -1
#         self.nodeLink = None
#         self.parent = none
#         self.children = [
#   class tree2-1:
#     def __init__(self, name, sup, parent):      [['A','B','C'],['C','A']]
#         self.name = 'A'
#         self.sup = 1
#         self.nodeLink = NONE
#         self.parent = parent
#         self.children = [
#                                     class tree3:
#                                         def __init__(self, name, sup, parent):
#                                             self.name = 'B'
#                                             self.sup = 1
#                                             self.nodeLink = None
#                                             self.parent = parent
#                                             self.children = [
#                                                                 class tree4:
#                                                                     def __init__(self, name, sup, parent):
#                                                                         self.name = 'B'
#                                                                         self.sup = 1
#                                                                         self.nodeLink = None
#                                                                         self.parent = parent
#                                                                         self.children = []
#                                                                 ]
#         ],
#   class tree2-2:
#     def __init__(self, name, sup, parent):
#         self.name = 'C'
#         self.sup = sup
#         self.nodeLink = TREE 4
#         self.parent = TREE1
#         self.children = []
# ]



#Function to get frequent itemsets with suffix 'node' and length n
def singlepath(node, n):
    c = 0
    sup = node.sup
    path = []
    pathname = []
    current = node
    
    #Get the path from current node to root
    while(current.parent != None): #查看目current是否為'root'，如果是則無parent，重複做直道回到最上層('root')
        path.append(current)     #將current內容紀錄(tree)
        pathname.append(current.name)      #('c')
        current = current.parent  #往上一層移動
    path.remove(node) #除了自己以外的路線
    pathname.remove(node.name) 
    candidatepath = []
    temp_candidatepath = []
   
    #Generate combinations of length n in the path
    a = (list(combinations(pathname, n))) #列出除了自己以外的長度為n的組合
    for j in a:
        temp_candidatepath.append(tuple(sorted(j)))   #以tuple形式存在temp_candidatapath裡
    #Append the suffix 'node.name' to the above paths
    for j in temp_candidatepath:
        j = list(j) #再次變回list
        j.append(node.name) #將原本剔除的當前TREE名加回去 變成包含當前的組合，就是在做freqent pattern
        candidatepath.append(sorted(j)) #在加回去變成 [['a','c'],['b','c']]
    #Update counts of the generated itemsets
    for j in candidatepath:
        j = tuple(j) #j = ('a','c')
        if j in count[n + 1]: #count[1]為符合minsup的字典，n至少會從1開始 
            count[n + 1][j] = count[n + 1][j] + sup  # 將之前已創好的value值，加上當前tree所有的sup
        else:
            count[n + 1][j] = sup #創建新字典,此sup為當前tree存有的sup
            
    #Iterating in the candidate tree recursively 
    if(node.nodeLink != None): #查看是否存在分支
        node=node.nodeLink
        singlepath(node, i) #i為組合長度
    
#Check if itemset is frequent
def frequent(n): #算出現次數
    f=0
    for i in count[n]:
        if(count[n][i] >= minsup):
            f = 1
    if(f == 1):
        return 1
    else:
        return 0

#Call singlepath function for all frequent nodes
for i in range(1, len(data.values[0]) + 1): #1列(ROW)有32的物品, +1是確保有32為一組的組合  #!
    if(frequent(i) == 1):
        for j in lastocc: #開始製造組合，將結果放到count[]裡，count唯有32空字典的list
            singlepath(lastocc[j], i)

#Remove infrequent itemsets
for z in range(len(data.values[0]) + 1):            
    for i in count[z].copy():
            if(count[z][i] < minsup): #查看所有的dict裡面的東西，是否符合minsup
                count[z].pop(i) #去掉不符合的

frequent_pattern = []
for items in count:
    for want in items:
        frequent_pattern.append(want)
                
#Get 'q', the length of the longest itemset
i=1 #設2是因為從第二筆的字典，才開始有東西
while(len(count[i]) != 0): #從這邊就能知道最大符合長度是多長，因為不符合的都被刪掉了，所以count最大的子矩陣
    i = i + 1
q = i - 1

#Find maximal and closed itemsets
maximal = []
closed = []
for i in range(1, q):
    for j in count[i]:
        fm = 0
        fc = 0
        for k in count[i + 1]:
            a = set(list([j]))
            b = set(list(k))
            #Set is maximal if no immediate superset is frequent
            if(a.intersection(b) == a): #如果兩者重疊則maximal
                fm = 1 
                #Set is closed if none of its immediate supersets have equal support
                if(count[i][j] == count[i + 1][k]): #出現次數相同
                    fc = 1
        if(fm == 0):
            maximal.append(j)
        if(fc == 0):
            closed.append(j)
#All sets at the top of the tree are automatically maximal and closed
for i in count[q]:
    maximal.append(i)
    closed.append(i)
print("frequent_pattern")
print('number of frequent pattern',len(frequent_pattern))
print(frequent_pattern)
 
#Find Association Rules 
print("Association_Rules")
all_frequent_pattern = {}
for len_FP in count:
    all_frequent_pattern.update(len_FP)
def rulegenerator(fitems): #fitmes = 最終dic
    '''
    Generates association rules from the frequent itemsets
    '''
    counter = 0
    
    for itemset in fitems.keys():
        if isinstance(itemset, str): #只有單個得我不要，我要tuple形式('a','b')
            continue
        length = len(itemset)#tuple裡面有幾項
        union_support = fitems[tuple(itemset)]#原本itemset為LIST形式，必須使用tuple(才可以變成key ，看兩個以上的品項的出現次數
        for i in range(1, length):
            lefts = map(list, combinations(itemset, i)) #['mineral water'], ['pancakes']
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]#單品項出現次數 
                        conf = union_support / leftcount
                if conf >= minconf:
                    right = list(itemset[:])
                    for e in left:                          #去除被分配在左邊的品項
                        right.remove(e)     
#                     fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')
                    print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
                    counter += 1
#                     fo.close()
    print(counter, "rules generated")

rulegenerator(all_frequent_pattern)


time_end = time.time()
print('FP_Growth_spent_time', time_end - time_start)
# print( time_end - time_start)
# print(len(ant))

Support-Threshold: 0.0045
Confidence-Threshold: 0.1
7501
frequent_pattern
number of frequent pattern 843
['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil', 'burgers', 'meatballs', 'eggs', 'turkey', 'milk', 'energy bar', 'whole wheat rice', 'whole wheat pasta', 'french fries', 'soup', 'light cream', 'shallot', 'frozen vegetables', 'spaghetti', 'pet food', 'cookies', 'cooking oil', 'champagne', 'chocolate', 'chicken', 'oil', 'fresh tuna', 'tomatoes', 'black tea', 'extra dark chocolate', 'protein bar', 'red wine', 'pasta', 'pepper', 'shampoo', 'rice', 'sparkling water', 'ham', 'body spray', 'pancakes', 'grated cheese', 'white wine', 'toothpaste', 'parmesan cheese', 'fresh bread', 'ground beef', 'escalope', 'herb & pepper', 'tomato sauce', 'magazines', 'strawb

In [5]:
list(all_frequent_pattern.keys())

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil',
 'burgers',
 'meatballs',
 'eggs',
 'turkey',
 'milk',
 'energy bar',
 'whole wheat rice',
 'whole wheat pasta',
 'french fries',
 'soup',
 'light cream',
 'shallot',
 'frozen vegetables',
 'spaghetti',
 'pet food',
 'cookies',
 'cooking oil',
 'champagne',
 'chocolate',
 'chicken',
 'oil',
 'fresh tuna',
 'tomatoes',
 'black tea',
 'extra dark chocolate',
 'protein bar',
 'red wine',
 'pasta',
 'pepper',
 'shampoo',
 'rice',
 'sparkling water',
 'ham',
 'body spray',
 'pancakes',
 'grated cheese',
 'white wine',
 'toothpaste',
 'parmesan cheese',
 'fresh bread',
 'ground beef',
 'escalope',
 'herb & pepper',
 'tomato sauce',
 'magazines',
 'strawberries',
 'strong cheese',
 'pickles',
 

In [4]:
FreqItems

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil',
 'burgers',
 'meatballs',
 'eggs',
 'turkey',
 'milk',
 'energy bar',
 'whole wheat rice',
 'whole wheat pasta',
 'french fries',
 'soup',
 'light cream',
 'shallot',
 'frozen vegetables',
 'spaghetti',
 'pet food',
 'cookies',
 'cooking oil',
 'champagne',
 'chocolate',
 'chicken',
 'oil',
 'fresh tuna',
 'tomatoes',
 'black tea',
 'extra dark chocolate',
 'protein bar',
 'red wine',
 'pasta',
 'pepper',
 'shampoo',
 'rice',
 'sparkling water',
 'ham',
 'body spray',
 'pancakes',
 'grated cheese',
 'white wine',
 'toothpaste',
 'parmesan cheese',
 'fresh bread',
 'ground beef',
 'escalope',
 'herb & pepper',
 'tomato sauce',
 'magazines',
 'strawberries',
 'strong cheese',
 'pickles',
 

In [6]:
def Diff(li1, li2): 
    return (list(set(li1).symmetric_difference(set(li2))))




print(Diff(list1, list2))

In [7]:
print(Diff(FreqItems, all_frequent_pattern))

[('ham', 'whole wheat rice')]
